# This notebook is for the creation of images which are then turned into videos

#### First import the required modules

Also grab useful globals and initialise matplotlib with nicer parameters

In [1]:
# Show plots in the browser
%matplotlib inline
#%matplotlib notebook

import matplotlib as mpl
import matplotlib.colors as col
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import numpy as np
#from datetime import datetime

# Display an image from file:
from IPython.display import Image
#Image(filename='test.png') 

from utilities import plotting, utils, fio




In [2]:
# Load the first sirivan output time step, this is used to show example plots
## Read sir ivan data
sirivan = fio.read_sirivan([fio._files_sirivan_[0]])
waroona = fio.read_waroona([fio._files_waroona_old_[0]])
#for k,v in data.items():
#    print(k, np.shape(v))

Reading  data/sirivan\umnsaa_pc2017021121.nc  ... 
Reading  data/waroona\umnsaa_pc2016010515.nc  ... 


AssertionError: topog latitude doesn't match pc file latitude

### AFAC Outline of weather at fire site

#### 211 topography, sites of interest, surface wind, and transect line

I can just plot all the sites I have since the map will only show those within the desired extent (boundaries E,W,S,N)

#### 223 vertical motion along transect

#### 224 horizontal wind speed along transect

Transects also show the topography as a black area along the bottom, Eventually we will add colour to this to depict fire heat flux (when I have fire data and I can read it)

In [7]:
def winds_2panel(topog, w, s, u, v, z, lat, lon, 
                 extentname='waroona',
                 transect=1, 
                 vectorskip=14,
                 quiverscale=60,
                 dtime=None,
                 ext='.png'
                ):
    '''
    211 Plot showing contour map and wind speed, along with near sites and transect
    223 plot showing vertical motion along transect
    224 plot showing wind speed along transect
    INPUTS:
        topography, vert motion, wind speed, z,lat,lon, 
        extentname = { 'waroona' | 'sirivan' } choice of two fire locations
        transect = int from 1 to 6 for transect choice
        vectorskip reduces quiver density (may need to fiddle)
        quiverscale changes how long the arrows are (also may need to fiddle)
        dtime is datetime of output 
        ext is the plot extension { '.png' | '.eps' }
    '''
    # datetime timestamp for file,title
    if dtime is None:
        dstamp = "20YYMMDDHHMM"
    else:
        dstamp = dtime.strftime("%Y%m%d%H%M")
    # figure name and location
    pname="figures/%s/winds_outline_X%d/fig_%s%s"%(extentname,transect,dstamp,ext)
    
    # set font sizes
    plotting.init_plots()
    
    # get plot extent, and transect
    extent = plotting._extents_[extentname]
    start,end = plotting._transects_["%s%d"%(extentname,transect)]
    
    plt.figure(figsize=[7,10])
    ax1 = plt.subplot(2,1,1)
    
    # top panel is topography
    plotting.map_topography(extent,topog,lat,lon)
    plt.title('Topography, wind speed ')
    
    # start to end x=[lon0,lon1], y=[lat0, lat1]
    plt.plot([start[1],end[1]],[start[0],end[0], ], '--k', 
             linewidth=2, marker='o', markersize=5)
    
    # add nearby towns
    if extent == 'waroona':
        plotting.map_add_locations(['waroona','yarloop'], 
                                   text=['Waroona', 'Yarloop'], 
                                   textcolor='lightgrey')
        # add fire ignition
        plotting.map_add_locations(['fire_waroona'],
                                   text = ['Fire ignition'], 
                                   color='r', marker='*', textcolor='lightgrey')
        # add pyroCB
    else:
        plotting.map_add_locations(['sirivan','uarbry'], 
                                   text=['Sir Ivan','Uarbry'],
                                   dx=[-.02,.05], dy =[-.015,-.03],
                                   textcolor='lightgrey')
        # add fire ignition
        plotting.map_add_locations(['fire_sirivan'],
                                   text = ['Fire ignition'], dx=.05,
                                   color='r', marker='*', textcolor='lightgrey')
        # add pyroCB

    
    # Add vectors for winds
    # just surface, and one every 10 points to reduce density
    skip = (slice(None,None,vectorskip),slice(None,None,vectorskip))
    #mlon,mlat = np.meshgrid(lon,lat)
    plt.quiver(lon[skip[1]],lat[skip[0]],u[0][skip],v[0][skip], scale=quiverscale)
    
    
    ## Second row is transect plots
    ax2 = plt.subplot(2,2,3)
    plotting.transect_w(w,z, lat, lon,start,end,topog=topog)
    plt.ylabel('height (m)')
    plt.xlabel('transect')
    
    ax3 = plt.subplot(2,2,4)
    plotting.transect_s(s,z,lat,lon,start,end,topog=topog)
    plt.yticks([])
    plt.xlabel('transect')
    
    # Save figure into animation folder with numeric identifier
    plt.suptitle(dstamp)
    print("SAVING FIGURE:",pname)
    plt.savefig(pname)
    plt.close()
    return pname


In [9]:
# Lets check the output
## EXAMPLE 

tstep=0
for data in [sirivan,waroona]:
    pname = winds_2panel(data['topog'], data['upward_air_velocity'][tstep], data['wind_speed'][tstep],
                         data['x_wind_destaggered'][tstep], data['y_wind_destaggered'][tstep],
                         data['zth'][tstep], data['latitude'], data['longitude'], 
                         extentname='sirivan')
    Image(filename=pname)


SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_20YYMMDDHHMM.png


array([413019.01666667, 413019.51666667])

## Now run this in a loop, creating one figure for each time step

The datestamp will be used to name each figure, hopefully ffmpeg can handle this

In [14]:
# Read each file one by one
for file_si in fio._files_sirivan_:
    data = fio.read_sirivan([file_si])
    # datetime of outputs
    dtime = utils.date_from_gregorian(data['time'])
    
    for tstep in range(2):
        pname = winds_2panel(data['topog'], data['upward_air_velocity'][tstep], data['wind_speed'][tstep],
                             data['x_wind_destaggered'][tstep], data['y_wind_destaggered'][tstep],
                             data['zth'][tstep], data['latitude'], data['longitude'], 
                             dtime=dtime[tstep],
                             extentname='sirivan')
    

Reading  data/sirivan/umnsaa_pc2017021203.nc  ... 
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702120300.png
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702120331.png
Reading  data/sirivan/umnsaa_pc2017021205.nc  ... 
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702120501.png
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702120531.png
Reading  data/sirivan/umnsaa_pc2017021202.nc  ... 
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702120201.png
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702120231.png
Reading  data/sirivan/umnsaa_pc2017021122.nc  ... 
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702112201.png
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702112230.png
Reading  data/sirivan/umnsaa_pc2017021123.nc  ... 
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702112301.png
SAVING FIGURE: figures/sirivan/winds_outline_X1/fig_201702112331.png
Reading  data/sirivan/umnsaa_pc2017021121.nc  ... 
SAVI

['data/sirivan/umnsaa_pc2017021121.nc', 'data/sirivan/umnsaa_pc2017021122.nc', 'data/sirivan/umnsaa_pc2017021123.nc', 'data/sirivan/umnsaa_pc2017021200.nc', 'data/sirivan/umnsaa_pc2017021201.nc', 'data/sirivan/umnsaa_pc2017021202.nc', 'data/sirivan/umnsaa_pc2017021203.nc', 'data/sirivan/umnsaa_pc2017021204.nc', 'data/sirivan/umnsaa_pc2017021205.nc']
